In [ ]:
import torch
import torchvision
from torchvision.models import ResNet18_Weights

# 1) Load a pretrained model (already trained; we're not training here)
model = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT).eval()

# 2) Dummy input shape must match the model's expectation
dummy_input = torch.randn(1, 3, 224, 224)

# 3) Export to ONNX: this freezes the graph + weights into a portable file
torch.onnx.export(
    model,
    dummy_input,
    "resnet18.onnx",
    input_names=["input"],       # name used later in TVM runtime
    output_names=["output"],
    opset_version=18,            # operator set version (compatibility)
    do_constant_folding=True,    # fold constants for small optimizations
    # dynamic_axes={"input": {0: "batch"}, "output": {0: "batch"}},  # optional
)

print("Wrote resnet18.onnx")

In [ ]:
import onnx
import tvm
from tvm import relay
from tvm.contrib import graph_executor

onnx_model = onnx.load("resnet18.onnx")

# Input shape
input_name = "input"
shape_dict = {input_name: (1, 3, 224, 224)}

# Import model to Relay
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

# Target: CPU (change to "cuda" for GPU)
target = "llvm"

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

# Save compiled artifacts
lib.export_library("deploy_lib_cpu.so")
with open("deploy_graph.json", "w") as f:
    f.write(lib.get_graph_json())
with open("deploy_params.params", "wb") as f:
    f.write(tvm.runtime.save_param_dict(lib.get_params()))

print("Compiled and exported TVM artifacts:")
print(" - deploy_lib_cpu.so")
print(" - deploy_graph.json")
print(" - deploy_params.params")